In [20]:
import math
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegressionCV
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_val_score, cross_val_predict, StratifiedKFold 
from sklearn import preprocessing, metrics, svm, ensemble
from sklearn.metrics import accuracy_score, classification_report
import tabpy_client

In [21]:
# Breast Cancer dataset
# Citation: Dr. William H. Wolberg, University of Wisconsin Hospitals, Madison 
# https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+(Original)

# Read the dataset (Note that the CSV provided for this demo has rows with the missing data removed)
df =  pd.read_csv('breastcancer.csv', header=0)

# Take a look at the structure of the file
df.head(n=10)


Id  Cl.thickness  Cell.size  Cell.shape  Marg.adhesion  Epith.c.size  \
0  1000025             5          1           1              1             2   
1  1002945             5          4           4              5             7   
2  1015425             3          1           1              1             2   
3  1016277             6          8           8              1             3   
4  1017023             4          1           1              3             2   
5  1017122             8         10          10              8             7   
6  1018099             1          1           1              1             2   
7  1018561             2          1           2              1             2   
8  1033078             2          1           1              1             2   
9  1033078             4          2           1              1             2   

   Bare.nuclei  Bl.cromatin  Normal.nucleoli  Mitoses      Class  
0            1            3                1        1     benign  
1           10            3                2        1     benign  
2            2            3                1        1     benign  
3            4            3                7        1     benign  
4            1            3                1        1     benign  
5           10            9                7        1  malignant  
6           10            3                1        1     benign  
7            1            3                1        1     benign  
8            1            1                1        5     benign  
9            1            2                1        1     benign

In [22]:
# Drop Id column not used in analysis
df.drop(['Id'], 1, inplace=True)

# Use LabelEncoder to convert textual classifications to numeric. 
# We will use the same encoder later to convert them back.
encoder = preprocessing.LabelEncoder()
df['Class'] = encoder.fit_transform(df['Class'])

# You could also do this manually in the following way:
# df['Class'] = df['Class'].map( {'benign': 0, 'malignant': 1} ).astype(int)

# Check the result of the transform
df.head(n=6)

Cl.thickness  Cell.size  Cell.shape  Marg.adhesion  Epith.c.size  \
0             5          1           1              1             2   
1             5          4           4              5             7   
2             3          1           1              1             2   
3             6          8           8              1             3   
4             4          1           1              3             2   
5             8         10          10              8             7   

   Bare.nuclei  Bl.cromatin  Normal.nucleoli  Mitoses  Class  
0            1            3                1        1      0  
1           10            3                2        1      0  
2            2            3                1        1      0  
3            4            3                7        1      0  
4            1            3                1        1      0  
5           10            9                7        1      1

In [23]:
# Split columns into independent/predictor variables vs dependent/response/outcome variable
X = np.array(df.drop(['Class'], 1))
y = np.array(df['Class'])

In [24]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [25]:
test= np.column_stack([X_test,y_test])
df = pd.DataFrame(test, columns= ['Cl_thickness', 'Cell_size', 'Cell_shape', 'Marg_adhesion', 'Epith_c_size', 
                     'Bare_nuclei', 'Bl_cromatin', 'Normal_nucleoli', 'Mitoses','Class'])
print(df)
df.to_csv (r'testbreast.csv', index = False, header=True)

     Cl_thickness  Cell_size  Cell_shape  Marg_adhesion  Epith_c_size  \
0               1          1           1              1             2   
1               3          1           1              1             2   
2               5          5           5              2             5   
3               4          7           8              3             4   
4               1          1           1              1             2   
5               3          3           1              1             2   
6               5          3           1              2             2   
7               8         10          10             10             7   
8               1          1           1              3             1   
9               1          1           1              1             2   
10              8          7           5             10             7   
11              4          1           1              1             2   
12             10          5           6           

In [26]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
print(X_test)

[[-1.22468404 -0.69781134 -0.74152574 ... -0.99628733 -0.62157783
  -0.33863738]
 [-0.51066644 -0.69781134 -0.74152574 ... -0.59244703 -0.62157783
  -0.33863738]
 [ 0.20335117  0.60365746  0.60468558 ...  0.21523357  0.0230214
  -0.33863738]
 ...
 [-0.15365763 -0.69781134 -0.74152574 ... -0.99628733 -0.62157783
  -0.33863738]
 [-0.15365763  2.23049348  0.26813275 ...  2.23443506  2.27911872
  -0.33863738]
 [-0.86767524 -0.69781134 -0.74152574 ... -0.59244703 -0.62157783
  -0.33863738]]


In [27]:
from sklearn.neural_network import MLPClassifier
classifier = MLPClassifier(alpha=1e-05, hidden_layer_sizes=(3, 2), random_state=1,
              solver='lbfgs')
classifier.fit(X_train,y_train)

MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(3, 2), learning_rate='constant',
              learning_rate_init=0.001, max_iter=200, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=1, shuffle=True, solver='lbfgs', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False)

In [28]:
y_pred= classifier.predict(X_test)
print(y_pred)

[0 0 1 1 0 0 0 1 0 0 1 0 1 0 0 1 1 1 1 0 0 0 1 0 1 1 0 0 0 1 0 1 1 0 0 0 1
 1 0 1 0 0 0 0 0 0 0 1 0 0 1 0 1 0 0 0 1 1 0 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0
 0 0 1 0 0 0 1 0 1 0 0 1 0 1 1 0 1 0 1 1 1 1 1 1 1 0 0 0 1 1 0 0 1 1 0 0 1
 0 0 1 0 0 0 0 0 0 0 1 0 0 1 1 0 1 0 1 0 0 1 0 0 1 0]


In [39]:
# Connect to TabPy server using the client library
connection = tabpy_client.Client('http://localhost:9004/')

In [30]:
# The scoring function that will use the Gradient Boosting Classifier to classify new data points
def Prediction(Cl_thickness, Cell_size, Cell_shape, Marg_adhesion, Epith_c_size, 
                     Bare_nuclei, Bl_cromatin, Normal_nucleoli, Mitoses):
    X = np.column_stack([Cl_thickness, Cell_size, Cell_shape, Marg_adhesion, Epith_c_size, 
                         Bare_nuclei, Bl_cromatin, Normal_nucleoli, Mitoses])
    X = sc.transform(X)
    return classifier.predict(X).tolist()

In [31]:
# Publish the SuggestDiagnosis function to TabPy server so it can be used from Tableau
# Using the name DiagnosticsDemo and a short description of what it does
connection.deploy('Test',
                  Prediction,
                  'Returns diagnosis suggestion based on ensemble model trained using Wisconsin Breast Cancer dataset', 
                  override=True)


In [53]:
Prediction(1,1,1,1,1,5,1,5,1)

[0]